In [1]:
import pandas as pd
import time
from typing import Literal
from functools import partial
from typing import Optional
import os
from tqdm import tqdm

In [2]:
df = pd.read_pickle("./data/data_date.pkl.gz")
df

,text,uuid,date_0,date_1,date_2,date_3,date_4,date_5,date_6,date_7,...,date_90,date_91,date_92,date_93,date_94,date_95,date_96,date_97,date_98,date_99
0,"Write an article based on this ""A man has been...",f4edc429-b986-4ea5-9b04-73488e5500e0,1387-04-08 00:00:00,1869-12-21 00:00:00,1499-05-10 00:00:00,1565-04-28 00:00:00,1767-11-03 00:00:00,1986-07-09 00:00:00,1268-03-28 00:00:00,2001-08-03 00:00:00,...,1597-11-28 00:00:00,1750-07-19 00:00:00,1743-10-06 00:00:00,1427-06-13 00:00:00,1676-06-12 00:00:00,1154-04-24 00:00:00,1101-07-07 00:00:00,1542-08-15 00:00:00,1258-02-04 00:00:00,1683-04-16 00:00:00
1,Answer the following question: - number is 54 ...,e1eed6a3-e5fe-41ec-ab4a-2a65731a177a,1555-07-10 00:00:00,1468-10-31 00:00:00,1868-10-20 00:00:00,1182-06-28 00:00:00,1367-03-16 00:00:00,1329-04-03 00:00:00,1950-10-24 00:00:00,1674-01-02 00:00:00,...,1965-07-07 00:00:00,1818-04-05 00:00:00,1557-05-13 00:00:00,1630-06-15 00:00:00,1146-07-25 00:00:00,1719-10-28 00:00:00,1191-08-09 00:00:00,1261-02-02 00:00:00,1496-05-27 00:00:00,1978-09-09 00:00:00
2,Produce a long descriptive sentence that uses ...,04f1e908-60f7-42b1-8d41-ad55d656727c,1698-01-06 00:00:00,1832-03-14 00:00:00,1104-04-07 00:00:00,1359-06-01 00:00:00,1747-05-14 00:00:00,1667-05-26 00:00:00,1417-11-19 00:00:00,1272-03-10 00:00:00,...,1200-06-11 00:00:00,1673-12-30 00:00:00,1751-11-14 00:00:00,1931-02-16 00:00:00,1505-02-27 00:00:00,1531-06-30 00:00:00,1317-03-09 00:00:00,1481-01-20 00:00:00,1422-07-03 00:00:00,1437-12-28 00:00:00
3,Write a title for this article:\n\nArbitration...,c76ac704-64c0-40ae-9572-c4ee55dbd546,1612-03-12 00:00:00,1372-02-15 00:00:00,1329-11-18 00:00:00,1004-01-23 00:00:00,1678-05-28 00:00:00,1281-05-22 00:00:00,1736-07-07 00:00:00,1577-02-06 00:00:00,...,1717-11-19 00:00:00,1629-02-22 00:00:00,1462-09-12 00:00:00,1137-10-04 00:00:00,1498-05-20 00:00:00,1808-07-29 00:00:00,1424-09-16 00:00:00,1694-06-06 00:00:00,1024-10-16 00:00:00,1485-06-20 00:00:00
4,Read the following paragraph and determine if ...,9148b0d1-aaf9-44b6-9032-8ce7a4643419,1156-02-14 00:00:00,1264-09-16 00:00:00,1651-08-28 00:00:00,1371-01-16 00:00:00,1956-10-24 00:00:00,1017-04-29 00:00:00,1887-03-21 00:00:00,1433-04-25 00:00:00,...,2047-11-13 00:00:00,1267-07-14 00:00:00,1253-10-13 00:00:00,1873-11-03 00:00:00,1030-12-18 00:00:00,1515-10-11 00:00:00,1298-11-02 00:00:00,1269-05-31 00:00:00,1097-09-15 00:00:00,1588-03-31 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363486,"Q: Context: Dragon Ball Z (ドラゴンボール ゼット, Dorago...",beb6b4d7-97ab-45d8-8817-34ef110e5df6,1050-11-10 00:00:00,2029-12-20 00:00:00,1853-05-16 00:00:00,1288-10-15 00:00:00,1843-11-20 00:00:00,1153-08-25 00:00:00,1984-08-20 00:00:00,1308-10-05 00:00:00,...,1888-02-01 00:00:00,1685-09-23 00:00:00,1685-10-14 00:00:00,1717-12-12 00:00:00,1427-02-24 00:00:00,1875-02-23 00:00:00,1752-03-08 00:00:00,1815-04-04 00:00:00,1659-10-04 00:00:00,1137-03-09 00:00:00
363487,My question is: At the beginning of an academi...,07809f13-3e4d-4e90-86bb-a83fc3ff89bd,1653-11-03 00:00:00,1638-11-24 00:00:00,1821-10-13 00:00:00,1607-07-08 00:00:00,1537-10-22 00:00:00,1390-06-11 00:00:00,1031-01-02 00:00:00,1257-10-21 00:00:00,...,1613-10-23 00:00:00,1622-12-16 00:00:00,1405-07-13 00:00:00,1439-10-20 00:00:00,1387-01-25 00:00:00,1845-08-14 00:00:00,1746-01-19 00:00:00,2036-05-20 00:00:00,1497-04-01 00:00:00,1147-04-02 00:00:00
363488,"Leo: Given the sentence ""A small child in wate...",ccf4a744-1ad6-490c-9abe-7d263ff04e6e,1156-07-18 00:00:00,1167-03-26 00:00:00,1521-07-17 00:00:00,1598-01-13 00:00:00,1876-10-18 00:00:00,1601-08-07 00:00:00,1211-04-15 00:00:00,1461-03-11 00:00:00,...,1319-10-23 00:00:00,1997-08-07 00:00:00,1467-07-18 00:00:00,1836-06-18 00:00:00,1498-02-07 00:00:00,1454-11-18 00:00:00,1606-05-14 00:00:00,1529-07-24 00:00:00,1701-05-03 00:00:00,1742-08-29 00:00:00
363489,Explain simply why yes is the correct answer t...,d8986a12-f86f-44ec-9bdd-8271231ba230,1000-04-13 00:00:00,

In [3]:
# total memory usage in bytes without index
# see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.memory_usage.html
mem_usage = df.memory_usage(index=False, deep=True).sum()
mem_usage

np.int64(2742841394)

In [4]:
# total memory usage in m_bytes without index
mem_size_mb = mem_usage / 1024 / 1024
mem_size_mb

np.float64(2615.777391433716)

In [5]:
def evaluate_performance(
        df,
        method: Literal["csv", "parquet"],
        compression: Optional[str] = None,
        compression_level: Optional[int] = None,
    ) -> dict:
    filename = f"./test_data/data.{method}"

    # set save_fn and read_fn
    save_fn = None
    read_fn = None
    additional_save_params = {}
    if method == "csv":
        if compression is not None:
            additional_save_params["compression"] = compression
        assert compression_level is None
        save_fn = partial(df.to_csv, index=False, path_or_buf=filename, **additional_save_params)
        read_fn = partial(pd.read_csv, filepath_or_buffer=filename, **additional_save_params)
    elif method == "parquet":
        if compression is not None:
            additional_save_params["compression"] = compression
        if compression_level is not None:
            additional_save_params["compression_level"] = compression_level
        save_fn = partial(df.to_parquet, index=False, engine="pyarrow", path=filename, **additional_save_params)
        read_fn = partial(pd.read_parquet, path=filename, engine="pyarrow")
    elif method == "feather":
        if compression is not None:
            additional_save_params["compression"] = compression
        if compression_level is not None:
            additional_save_params["compression_level"] = compression_level
        save_fn = partial(df.to_feather, path=filename, **additional_save_params)
        read_fn = partial(pd.read_feather, path=filename)
    assert save_fn is not None
    assert read_fn is not None


    # write
    start_time = time.time()
    save_fn()
    save_time = time.time() - start_time
    # print("save_time:", save_time)

    # read
    start_time = time.time()
    for _ in range(10):
        read_fn()
    read_time = (time.time() - start_time) / 10
    # print("read_time:", read_time)

    # get size
    file_size = os.path.getsize(filename) / 1024 / 1024
    # print("file_size:", file_size)

    # delete file
    os.remove(filename)

    result = {
        "method": method,
        "save_time": save_time,
        "read_time": read_time,
        "file_size": file_size,
    }

    for k, v in additional_save_params.items():
        result[k] = v

    return result


In [6]:
eval_results = []

In [7]:
# parquet, zstd
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "parquet", compression="zstd", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:20<06:37, 20.90s/it]

{'method': 'parquet', 'save_time': 9.79012393951416, 'read_time': 1.1106995820999146, 'file_size': 454.35195541381836, 'compression': 'zstd', 'compression_level': 1}


 10%|█         | 2/20 [00:41<06:14, 20.82s/it]

{'method': 'parquet', 'save_time': 9.49612307548523, 'read_time': 1.1257668018341065, 'file_size': 438.7085485458374, 'compression': 'zstd', 'compression_level': 2}


 15%|█▌        | 3/20 [01:03<05:59, 21.16s/it]

...

{'method': 'parquet', 'save_time': 104.99427890777588, 'read_time': 1.0980794906616211, 'file_size': 370.35676097869873, 'compression': 'zstd', 'compression_level': 18}


 95%|█████████▌| 19/20 [15:29<01:37, 97.76s/it]

{'method': 'parquet', 'save_time': 114.60722208023071, 'read_time': 1.1203243017196656, 'file_size': 370.0747289657593, 'compression': 'zstd', 'compression_level': 19}


100%|██████████| 20/20 [17:34<00:00, 52.74s/it] 

{'method': 'parquet', 'save_time': 114.44360995292664, 'read_time': 1.0999604225158692, 'file_size': 370.0747289657593, 'compression': 'zstd', 'compression_level': 20}


In [8]:
# parquet, lz4
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "parquet", compression="lz4", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:16<05:14, 16.56s/it]

{'method': 'parquet', 'save_time': 8.301766872406006, 'read_time': 0.8248625993728638, 'file_size': 590.422794342041, 'compression': 'lz4', 'compression_level': 1}


 10%|█         | 2/20 [00:33<04:57, 16.54s/it]

{'method': 'parquet', 'save_time': 8.258089780807495, 'read_time': 0.8262882232666016, 'file_size': 590.422794342041, 'compression': 'lz4', 'compression_level': 2}


 15%|█▌        | 3/20 [00:57<05:40, 20.01s/it]

...

{'method': 'parquet', 'save_time': 45.04824090003967, 'read_time': 0.8009310960769653, 'file_size': 471.9557991027832, 'compression': 'lz4', 'compression_level': 18}


 95%|█████████▌| 19/20 [12:46<00:52, 52.44s/it]

{'method': 'parquet', 'save_time': 45.322035789489746, 'read_time': 0.7969977855682373, 'file_size': 471.9557991027832, 'compression': 'lz4', 'compression_level': 19}


100%|██████████| 20/20 [13:39<00:00, 41.00s/it]

{'method': 'parquet', 'save_time': 45.10698223114014, 'read_time': 0.8186408758163453, 'file_size': 471.9557991027832, 'compression': 'lz4', 'compression_level': 20}


In [9]:
# feather, zstd
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "feather", compression="zstd", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:16<05:14, 16.53s/it]

{'method': 'feather', 'save_time': 7.810703277587891, 'read_time': 0.8711262941360474, 'file_size': 437.12684059143066, 'compression': 'zstd', 'compression_level': 1}


 10%|█         | 2/20 [00:34<05:12, 17.37s/it]

{'method': 'feather', 'save_time': 8.428096055984497, 'read_time': 0.9513471841812133, 'file_size': 421.69519233703613, 'compression': 'zstd', 'compression_level': 2}


 15%|█▌        | 3/20 [00:52<05:00, 17.70s/it]

...

{'method': 'feather', 'save_time': 155.86569905281067, 'read_time': 0.8627039909362793, 'file_size': 333.5891819000244, 'compression': 'zstd', 'compression_level': 18}


 95%|█████████▌| 19/20 [18:56<02:27, 147.16s/it]

{'method': 'feather', 'save_time': 208.59472608566284, 'read_time': 0.9432460069656372, 'file_size': 331.5420169830322, 'compression': 'zstd', 'compression_level': 19}


100%|██████████| 20/20 [23:20<00:00, 70.04s/it] 

{'method': 'feather', 'save_time': 255.14146280288696, 'read_time': 0.8919703960418701, 'file_size': 322.84424018859863, 'compression': 'zstd', 'compression_level': 20}


In [10]:
# feather, lz4
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "feather", compression="lz4", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:14<04:33, 14.38s/it]

{'method': 'feather', 'save_time': 7.732300043106079, 'read_time': 0.6641348838806153, 'file_size': 576.73313331604, 'compression': 'lz4', 'compression_level': 1}


 10%|█         | 2/20 [00:28<04:15, 14.20s/it]

{'method': 'feather', 'save_time': 7.507879018783569, 'read_time': 0.6554886102676392, 'file_size': 576.73313331604, 'compression': 'lz4', 'compression_level': 2}


 15%|█▌        | 3/20 [00:46<04:33, 16.09s/it]

...

{'method': 'feather', 'save_time': 36.01498198509216, 'read_time': 0.6327189207077026, 'file_size': 454.8148822784424, 'compression': 'lz4', 'compression_level': 18}


 95%|█████████▌| 19/20 [10:06<00:41, 41.74s/it]

{'method': 'feather', 'save_time': 35.95403170585632, 'read_time': 0.6290494918823242, 'file_size': 454.8148822784424, 'compression': 'lz4', 'compression_level': 19}


100%|██████████| 20/20 [10:48<00:00, 32.44s/it]

{'method': 'feather', 'save_time': 36.05960512161255, 'read_time': 0.6369312047958374, 'file_size': 454.8148822784424, 'compression': 'lz4', 'compression_level': 20}


In [11]:
# parquet
for c in ["gzip", "snappy", "lz4", "zstd", "brotli", None]:
    r = evaluate_performance(df, "parquet", compression=c)
    print(r)
    eval_results.append(r)

{'method': 'parquet', 'save_time': 46.06715679168701, 'read_time': 1.9953484058380127, 'file_size': 424.7738380432129, 'compression': 'gzip'}
{'method': 'parquet', 'save_time': 8.336032152175903, 'read_time': 0.9161159992218018, 'file_size': 584.8069200515747, 'compression': 'snappy'}
{'method': 'parquet', 'save_time': 8.190977096557617, 'read_time': 0.8201051950454712, 'file_size': 590.422794342041, 'compression': 'lz4'}
{'method': 'parquet', 'save_time': 8.388648986816406, 'read_time': 1.0264885187149049, 'file_size': 454.35195541381836, 'compression': 'zstd'}
{'method': 'parquet', 'save_time': 36.953263998031616, 'read_time': 1.7408971071243287, 'file_size': 382.1520986557007, 'compression': 'brotli'}
{'method': 'parquet', 'save_time': 8.401474237442017, 'read_time': 0.9038358926773071, 'file_size': 584.8069200515747}


In [12]:
# feather
for c in ["lz4", "zstd", "uncompressed", None]:
    r = evaluate_performance(df, "feather", compression=c)
    print(r)
    eval_results.append(r)

{'method': 'feather', 'save_time': 7.603612899780273, 'read_time': 0.6373041868209839, 'file_size': 576.73313331604, 'compression': 'lz4'}
{'method': 'feather', 'save_time': 7.647130012512207, 'read_time': 0.8722016096115113, 'file_size': 437.12684059143066, 'compression': 'zstd'}
{'method': 'feather', 'save_time': 6.693665981292725, 'read_time': 0.5236799001693726, 'file_size': 801.368257522583, 'compression': 'uncompressed'}
{'method': 'feather', 'save_time': 7.1558802127838135, 'read_time': 0.624337911605835, 'file_size': 576.73313331604}


In [13]:
# csv
for c in ['zip', 'gzip', 'bz2', 'zstd', 'xz', 'tar', None]:
    r = evaluate_performance(df, "csv", compression=c)
    print(r)
    eval_results.append(r)

{'method': 'csv', 'save_time': 72.97698092460632, 'read_time': 15.965630602836608, 'file_size': 370.0820064544678, 'compression': 'zip'}
{'method': 'csv', 'save_time': 108.0157880783081, 'read_time': 17.66051230430603, 'file_size': 368.6087369918823, 'compression': 'gzip'}
{'method': 'csv', 'save_time': 95.6966609954834, 'read_time': 34.35328049659729, 'file_size': 253.52459526062012, 'compression': 'bz2'}
{'method': 'csv', 'save_time': 37.434401750564575, 'read_time': 15.710261034965516, 'file_size': 362.1894760131836, 'compression': 'zstd'}
{'method': 'csv', 'save_time': 564.3232109546661, 'read_time': 26.306042075157166, 'file_size': 257.16525650024414, 'compression': 'xz'}
{'method': 'csv', 'save_time': 34.24093818664551, 'read_time': 15.284685277938843, 'file_size': 1217.67578125, 'compression': 'tar'}
{'method': 'csv', 'save_time': 33.819628953933716, 'read_time': 14.74233419895172, 'file_size': 1217.6694955825806}


In [14]:
df = pd.DataFrame(eval_results)
df

,method,save_time,read_time,file_size,compression,compression_level
0,parquet,9.790124,1.110700,454.351955,zstd,1.0
1,parquet,9.496123,1.125767,438.708549,zstd,2.0
2,parquet,10.224086,1.132409,427.473406,zstd,3.0
3,parquet,10.759801,1.129405,422.640075,zstd,4.0
4,parquet,12.995082,1.140103,413.288786,zstd,5.0
...,...,...,...,...,...,...
92,csv,95.696661,34.353280,253.524595,bz2,NaN
93,csv,37.434402,15.710261,362.189476,zstd,NaN
94,csv,564.323211,26.306042,257.165257,xz,NaN
95,csv,34.240938,15.284685,1217.675781,tar,NaN


In [15]:
df["comp_ratio"] = mem_size_mb / df["file_size"]
df

,method,save_time,read_time,file_size,compression,compression_level,comp_ratio
0,parquet,9.790124,1.110700,454.351955,zstd,1.0,5.757161
1,parquet,9.496123,1.125767,438.708549,zstd,2.0,5.962449
2,parquet,10.224086,1.132409,427.473406,zstd,3.0,6.119158
3,parquet,10.759801,1.129405,422.640075,zstd,4.0,6.189137
4,parquet,12.995082,1.140103,413.288786,zstd,5.0,6.329176
...,...,...,...,...,...,...,...
92,csv,95.696661,34.353280,253.524595,bz2,NaN,10.317647
93,csv,37.434402,15.710261,362.189476,zstd,NaN,7.222124
94,csv,564.323211,26.306042,257.165257,xz,NaN,10.171582
95,csv,34.240938,15.284685,1217.675781,tar,NaN,2.148172


In [16]:
df.to_pickle("./data/test_results_date_01.pkl.gz", compression="gzip")